#### Reading Stocks

In [3]:
# loading packages (pip/pip3 list for new packages)
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
import akshare as ak 

In [33]:
df_raw = ak.stock_zh_a_spot()

In [35]:
df_raw.head

<bound method NDFrame.head of             代码    名称     最新价   涨跌额    涨跌幅      买入      卖出      昨收      今开  \
0     bj430017  星昊医药   13.10 -0.74 -5.347   13.09   13.10   13.84   13.65   
1     bj430047  诺思兰德   14.97 -0.47 -3.044   14.97   15.03   15.44   15.56   
2     bj430090  同辉信息    3.66 -0.19 -4.935    3.65    3.66    3.85    3.81   
3     bj430139  华岭股份   12.33 -0.50 -3.897   12.33   12.34   12.83   12.79   
4     bj430198  微创光电    9.18 -0.58 -5.943    9.18    9.19    9.76    9.79   
...        ...   ...     ...   ...    ...     ...     ...     ...     ...   
5346  sz301568   思泰克   38.81  1.89  5.119   38.81   38.82   36.92   36.87   
5347  sz301577  美信科技   60.10  4.32  7.745   60.10   60.11   55.78   55.00   
5348  sz301578  辰奕智能   54.51  2.08  3.967   54.51   54.52   52.43   51.37   
5349  sz301589  诺瓦星云  431.90  3.75  0.876  431.90  432.00  428.15  423.00   
5350  sz301591   C肯特   55.32 -3.46 -5.886   55.32   55.33   58.78   57.65   

          最高      最低        成交量          成交额 

In [48]:
df = df_raw[['代码','名称']].copy()
df.rename(columns={'代码': 'symbol', '名称': 'name'}, inplace=True)
symbol_name_dict = df.set_index('symbol')['name'].to_dict()
# symbol_name_dict.get('sh600029')

In [49]:
today = datetime.now().date().strftime("%Y-%m-%d")
start_date, end_date ="2024-01-01",today
stock_list = ['sh600887','sh600009','sh600029','sh600004','sz000651']

def get_signal(stock_index):

    stock_data = ak.stock_zh_a_daily(symbol=stock_index, start_date=start_date, end_date=end_date)

    # 计算5日和20日均线
    stock_data["ma5"] = stock_data["close"].rolling(window=5).mean()
    stock_data["ma20"] = stock_data["close"].rolling(window=20).mean()

    # 生成买入信号（ma5向上穿越ma20）
    stock_data["buy_signal"] = (stock_data["ma5"] > stock_data["ma20"]) & (stock_data["ma5"].shift(1) <= stock_data["ma20"].shift(1))

    # 生成卖出信号（ma5向下穿越ma20）
    stock_data["sell_signal"] = (stock_data["ma5"] < stock_data["ma20"]) & (stock_data["ma5"].shift(1) >= stock_data["ma20"].shift(1))

    # 打印交易信号
    signals = stock_data[["date", "buy_signal", "sell_signal"]]
    # print(signals)

    return stock_data

In [53]:
for stock_index in stock_list:
    stock_data = get_signal(stock_index)
    print(f'{symbol_name_dict.get(stock_index)}: {stock_index}\n')

伊利股份: sh600887

上海机场: sh600009

南方航空: sh600029

白云机场: sh600004

格力电器: sz000651



##### Visualisation

In [14]:
# 获取A股指数数据
a_share_data = ak.stock_zh_index_daily(index="sz000001", start_date=start_date, end_date=end_date)

# 获取美股指数数据
us_stock_data = ak.stock_us_daily(symbol="DJI", start_date=start_date, end_date=end_date)

# 绘制A股和美股指数比较图
plt.figure(figsize=(12, 6))
plt.plot(a_share_data["date"], a_share_data["close"], label="A股指数")
plt.plot(us_stock_data["date"], us_stock_data["close"], label="美股指数")
plt.xlabel("Date")
plt.ylabel("Close Price")
plt.title("A股和美股指数比较")
plt.legend()
plt.show()

TypeError: stock_zh_index_daily() got an unexpected keyword argument 'index'

##### # read more on https://zhuanlan.zhihu.com/p/678521592